# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file from Part I WeatherPy and Store in a DataFrame
file = "output_data/Weather_by_City.csv"
weather_data = pd.read_csv(file)
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint George,37.1041,-113.5841,93.79,27,20,4.00,US,1627872506
1,1,Hithadhoo,-0.6000,73.0833,81.27,73,83,6.87,MV,1627872679
2,2,New Norfolk,-42.7826,147.0587,54.90,55,67,5.23,AU,1627872680
3,3,Selizharovo,56.8519,33.4487,59.02,74,97,3.85,RU,1627872680
4,4,Yerbogachën,61.2767,108.0108,72.03,44,100,9.80,RU,1627872277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Congifure gmaps
gmaps.configure(api_key=g_key)

# Coordinates of the weather data
locations = weather_data[["Lat", "Lng"]]
weight = weather_data["Humidity"]

In [4]:
# Draw the map
figure_layout = {"width": "400px", "height": "300px", "border": "1px solid black", "padding": "1px",
                 "margin": "0 auto 0 auto"}

fig = gmaps.figure(figure_layout, center=(10,0), zoom_level=2)

# Add a heatmap
max_humidity = weather_data["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations, weight)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# I want to go somewhere with 0-30 % humidity, max temp 85 F, and wind speed less than 7 mph

best_vacation = weather_data.loc[(weather_data["Humidity"] < 30) & (weather_data["Max Temp"] < 85) &
                                (weather_data["Wind Speed"] < 7)] #9 places
best_vacation = best_vacation.dropna(how="any")
best_vacation = best_vacation.reset_index() 
best_vacation # no null values -- 9 rows

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,96,96,Kumul,42.8000,93.4500,78.37,25,4,2.33,CN,1627872756
1,108,108,Khandyga,62.6667,135.6000,83.91,21,0,4.90,RU,1627872798
2,120,120,Zharkent,44.1628,80.0000,83.07,15,3,5.32,KZ,1627872808
3,175,175,Korla,41.7597,86.1469,80.31,16,73,6.96,CN,1627872874
4,221,221,Ubari,26.5921,12.7805,82.98,13,0,4.41,LY,1627872933
5,245,245,Taloqan,36.7361,69.5345,84.79,18,0,2.08,AF,1627872946
6,348,348,Assiut,27.1810,31.1837,80.78,22,9,3.11,EG,1627873064
7,472,472,Mayor Pablo Lagerenza,-19.9309,-60.7718,63.37,29,100,4.07,PY,1627873231
8,550,550,Ivanovka,42.8878,75.0850,75.63,13,82,2.01,KG,1627873335


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# Store the above DataFrame into hotel_df and drop the index and Unnamed columns
hotel_df = best_vacation.drop(["index", "Unnamed: 0"], axis=1)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kumul,42.8000,93.4500,78.37,25,4,2.33,CN,1627872756
1,Khandyga,62.6667,135.6000,83.91,21,0,4.90,RU,1627872798
2,Zharkent,44.1628,80.0000,83.07,15,3,5.32,KZ,1627872808
3,Korla,41.7597,86.1469,80.31,16,73,6.96,CN,1627872874
4,Ubari,26.5921,12.7805,82.98,13,0,4.41,LY,1627872933
5,Taloqan,36.7361,69.5345,84.79,18,0,2.08,AF,1627872946
6,Assiut,27.1810,31.1837,80.78,22,9,3.11,EG,1627873064
7,Mayor Pablo Lagerenza,-19.9309,-60.7718,63.37,29,100,4.07,PY,1627873231
8,Ivanovka,42.8878,75.0850,75.63,13,82,2.01,KG,1627873335


In [8]:
# Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Lodging"
        
hotel_list = []

x = 0

for city in hotel_df["City"]:
    target_coordinates = str(hotel_df["Lat"][x]) + ', ' + str(hotel_df["Lng"][x])
    x = x + 1
    
    params={"location":target_coordinates, "keyword":target_search, "radius":target_radius, "type":target_type,
           "key":g_key}
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_list.append(hotel_name)
    
    except IndexError:
        hotel_list.append(np.nan)
    
hotel_list

['哈密环球大酒店',
 'Prezidentskaya Gostinitsa',
 'гостиница Satti',
 'The Garden Hotel Korla',
 nan,
 'Hotel starai Pamirتالار عروسی ستاره پامیر',
 'Assiut hotels Armed Forces',
 nan,
 nan]

In [28]:
# Add the hotel_list to the hotel_df DataFrame
hotel_df["Hotel Name"] = hotel_list
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kumul,42.8000,93.4500,78.37,25,4,2.33,CN,1627872756,哈密环球大酒店
1,Khandyga,62.6667,135.6000,83.91,21,0,4.90,RU,1627872798,Prezidentskaya Gostinitsa
2,Zharkent,44.1628,80.0000,83.07,15,3,5.32,KZ,1627872808,гостиница Satti
3,Korla,41.7597,86.1469,80.31,16,73,6.96,CN,1627872874,The Garden Hotel Korla
4,Ubari,26.5921,12.7805,82.98,13,0,4.41,LY,1627872933,NaN
5,Taloqan,36.7361,69.5345,84.79,18,0,2.08,AF,1627872946,Hotel starai Pamirتالار عروسی ستاره پامیر
6,Assiut,27.1810,31.1837,80.78,22,9,3.11,EG,1627873064,Assiut hotels Armed Forces
7,Mayor Pablo Lagerenza,-19.9309,-60.7718,63.37,29,100,4.07,PY,1627873231,NaN
8,Ivanovka,42.8878,75.0850,75.63,13,82,2.01,KG,1627873335,NaN


In [31]:
# Clean the DataFrame by removing NaN values
hotel_df = hotel_df.dropna(how="any")
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kumul,42.8000,93.4500,78.37,25,4,2.33,CN,1627872756,哈密环球大酒店
1,Khandyga,62.6667,135.6000,83.91,21,0,4.90,RU,1627872798,Prezidentskaya Gostinitsa
2,Zharkent,44.1628,80.0000,83.07,15,3,5.32,KZ,1627872808,гостиница Satti
3,Korla,41.7597,86.1469,80.31,16,73,6.96,CN,1627872874,The Garden Hotel Korla
4,Taloqan,36.7361,69.5345,84.79,18,0,2.08,AF,1627872946,Hotel starai Pamirتالار عروسی ستاره پامیر
5,Assiut,27.1810,31.1837,80.78,22,9,3.11,EG,1627873064,Assiut hotels Armed Forces


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
marker_coordinates = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_coordinates)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))